In [1]:
# imports
import pandas as pd
import sklearn as skl
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
# SQL query
data_sql = """
SELECT *
FROM final_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)
data_df.head()

,GUID,Name,HallOfFameStatus,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,Team,...,BLK,TOV,EFF,idPlayer,numberRound,BIRTHDATE,POSITION,AGE_ROOKIE_SEASON,Draft_Decade,numBallotsBeforeInduct
0,MITCHMCGARY2014,Mitch McGary,Not Inducted,NaN,2014,2015,2,2020,21,OKC,...,0.5,1.0,9.0,203956.0,1,1992-06-06,Forward,22.36,2010s,NaN
1,PJHAIRSTON2014,PJ Hairston,Not Inducted,NaN,2014,2015,2,2020,26,MIA,...,0.3,0.5,4.3,203798.0,1,1992-12-24,Forward,21.81,2010s,NaN
2,KJMCDANIELS2014,KJ McDaniels,Not Inducted,NaN,2014,2016,3,2021,32,PHI,...,1.1,1.7,7.6,203909.0,2,1993-02-09,Guard,21.68,2010s,NaN
3,CLEANTHONYEARLY2014,Cleanthony Early,Not Inducted,NaN,2014,2015,2,2020,34,NYK,...,0.3,1.0,4.8,203921.0,2,1991-04-17,Forward,23.50,2010s,NaN
4,CORYJEFFERSON2014,Cory Jefferson,Not Inducted,NaN,2014,2015,2,2020,60,SAS,...,0.4,0.4,4.8,203928.0,2,1990-12-26,Forward,23.80,2010s,NaN


### Preprocessing 

In [4]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column, drop_first no ideal

# drop coulmns/features
data_df.drop(['GUID',
              'Name',
              'HallofFameClass',
              'YearDrafted',
              'TO_YEAR',
              'Years_Played',
              'HOF_Elgibility_Year',
              'Team',
              'College',
              'GP',
              'MIN',
              'idPlayer',
              'BIRTHDATE',
              'POSITION',
              'AGE_ROOKIE_SEASON',
              'Draft_Decade',
              'numBallotsBeforeInduct'], 
             axis=1, inplace=True)

data_df.head()
print(data_df.shape)

,Pick,PTS,FGM,FGA,FG%,3P_Made,3PA,3P%,FTM,FTA,...,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,numberRound,HOF_Hall of Fame Member
0,21,6.3,2.8,5.2,53.3,0.0,0.1,0.0,0.8,1.3,...,1.7,3.5,5.2,0.4,0.5,0.5,1.0,9.0,1,0
1,26,5.6,1.9,6.0,32.3,1.1,3.6,30.1,0.7,0.8,...,0.5,1.6,2.0,0.5,0.5,0.3,0.5,4.3,1,0
2,32,7.9,2.8,7.0,39.6,0.7,2.4,28.7,1.6,2.1,...,1.0,2.2,3.2,1.2,0.7,1.1,1.7,7.6,2,0
3,34,5.4,1.9,5.5,35.5,0.6,2.2,26.2,0.9,1.2,...,0.6,1.9,2.5,0.9,0.6,0.3,1.0,4.8,2,0
4,60,3.7,1.5,3.3,44.9,0.0,0.3,13.3,0.6,1.1,...,0.9,2.0,2.9,0.3,0.2,0.4,0.4,4.8,2,0


(1217, 21)


In [5]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from path import Path
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf

In [6]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

In [7]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
Counter(y_train)

Counter({0: 880, 1: 32})

### Decision Tree Scaled

In [8]:
# create a StandardScaler instance
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# create the decision tree classifier instance.
model = tree.DecisionTreeClassifier()

# fit the model.
model = model.fit(X_train_scaled, y_train)

In [10]:
# DT predictions
predictions = model.predict(X_test_scaled)

# calcuate the accuracy score.
test_acc_score = accuracy_score(y_test, predictions)
train_acc_score = model.score(X_train, y_train)

In [11]:
# define confusion matrix
cm = confusion_matrix(y_test, predictions)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

In [12]:
# DT results
print('DECISION TREE SCALED RESULTS\n')
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report(y_test, predictions))

DECISION TREE SCALED RESULTS

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,278,17
HOF Member,8,2


ACCURACY SCORES

Training Accuracy: 0.4276315789473684

Testing Accuracy: 0.9180327868852459

CLASSIFICATION REPORT

              precision    recall  f1-score   support

           0       0.97      0.94      0.96       295
           1       0.11      0.20      0.14        10

    accuracy                           0.92       305
   macro avg       0.54      0.57      0.55       305
weighted avg       0.94      0.92      0.93       305

